In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_excel('data/dataset.xlsx', engine="openpyxl")
data = data.dropna() # drop missing values
data.head()

,Num,Timestamp,Current_J0,Temperature_T0,Current_J1,Temperature_J1,Current_J2,Temperature_J2,Current_J3,Temperature_J3,...,Speed_J0,Speed_J1,Speed_J2,Speed_J3,Speed_J4,Speed_J5,Tool_current,cycle,Robot_ProtectiveStop,grip_lost
0,1,2022-10-26T08:17:21.847Z,0.109628,27.875,-2.024669,29.3750,-1.531442,29.3750,-0.998570,32.1250,...,2.955651e-01,-0.000490,0.001310,-0.132836,-0.007479,-0.152962,0.082732,1,0.0,False
1,2,2022-10-26T08:17:22.852Z,0.595605,27.875,-2.278456,29.3125,-0.866556,29.4375,-0.206097,32.1875,...,-7.391485e-30,-0.000304,0.002185,0.001668,-0.000767,0.000417,0.505895,1,0.0,False
2,3,2022-10-26T08:17:23.857Z,-0.229474,27.875,-2.800408,29.3125,-2.304336,29.4375,-0.351499,32.1250,...,1.369386e-01,0.007795,-2.535874,0.379867,0.000455,-0.496856,0.079420,1,0.0,False
3,4,2022-10-26T08:17:24.863Z,0.065053,27.875,-3.687768,29.3125,-1.217652,29.4375,-1.209115,32.1250,...,-9.030032e-02,-0.004911,-0.009096,-0.384196,0.018411,0.425559,0.083325,1,0.0,False
4,5,2022-10-26T08:17:25.877Z,0.884140,27.875,-2.938830,29.3750,-1.794076,29.4375,-2.356471,32.1875,...,1.268088e-01,0.005567,0.001138,-0.353284,0.014994,0.180989,0.086379,1,0.0,False


In [3]:
# Feature columns, but don't use Timestamp and Num
feature_columns = [
    'Current_J0', 'Temperature_T0', 'Current_J1', 'Temperature_J1',
    'Current_J2', 'Temperature_J2', 'Current_J3', 'Temperature_J3',
    'Current_J4', 'Temperature_J4', 'Current_J5', 'Temperature_J5',
    'Speed_J0', 'Speed_J1', 'Speed_J2', 'Speed_J3',
    'Speed_J4', 'Speed_J5', 'Tool_current', 'cycle '
]

# Features and labels
X = data[feature_columns].values
y = data['grip_lost'].values

In [4]:
# Turn boolean variables to binary values
y = np.where(y == False, 0, 1)
print(y[:400])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0]


In [5]:
# Normalizing
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

In [6]:
# Shuffle data
indices = np.arange(X.shape[0])
np.random.shuffle(indices)

# Split index 80/20 ratio between train and test 
split_idx = int(0.8 * len(indices))

X_train, X_test = X[indices[:split_idx]], X[indices[split_idx:]]
y_train, y_test = y[indices[:split_idx]], y[indices[split_idx:]]

In [7]:
def rbf_kernel(x1, x2, gamma):
    dist = np.linalg.norm(x1-x2)**2
    return np.exp(-gamma * dist)

In [8]:
# Info https://scikit-learn.org/dev/auto_examples/svm/plot_rbf_parameters.html
class svm_rbf:
    def __init__(self, C=1.0, gamma=0.5, tol=1e-3, max_passes=5):
        self.C = C
        self.gamma = gamma
        self.tol = tol
        self.max_passes = max_passes
        self.alphas = None
        self.b = 0
        self.X = None
        self.y = None
        self.K = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.X = X
        self.y = y
        self.alphas = np.zeros(n_samples) # init numpy array with zeros
        self.b = 0

        self.K = np.zeros((n_samples, n_samples)) 

        for i in range(n_samples):
            for j in range(n_samples):
                self.K[i,j] = rbf_kernel(X[i], X[j], self.gamma)

        passes = 0
        while passes < self.max_passes:
            num_changed_alphas = 0
            for i in range(n_samples):
                error_i = self._decision_func_single(X[i] - y[i])

                first_cond = y[i]*error_i < - self.tol and self.alphas < self.C
                second_cond = y[i]*error_i > self.tol and self.alphas[i] > 0

                if (first_cond or second_cond):

                    # select second alpha j
                    j = np.random.choice([x for x in range(n_samples) if x != i])
                    error_j = self._decision_func_single(x[j] - y[j])

                    alpha_i_old = self.alphas[i]
                    alpha_j_old = self.alphas[j]

                    # bounds for L and H for alpha j so it stays in [0, C]
                    if y[i] != y[j]:
                        L = max(0, self.alphas[j] - self.alphas[i])
                        H = min(self.C, self.C + self.alphas[j] - self.alphas[i])

                    else:
                        L = max(0, self.alphas[i] + self.alphas[j] - self.C)
                        H = min(self.C, self.alphas[i] + self.alphas[j])

                    # Skip updated because no optimaztion possible
                    if L == H:
                        continue

                    # η=K(x_i, x_i) +  K(x_j, x_j) - 2K(x_i, x_j)
                    eta = 2*self.K[i, j] - self.K[i, i] - self.K[j, j]
                    # If eta is larger than zero it increases the objective function which is the opposite of what we want
                    if eta >= 0:
                        continue
                    
                    #update new alphas_j and clip them in range L, H
                    self.alphas[j] -= y[j] * (E_i - E_j) / eta
                    self.alphas[j] = np.clip(self.alphas[j], L, H)

                    # dont update if diff below threshold of 1e-5
                    if abs(self.alphas[j] - alpha_j_old) < 1e-5:
                        continue
                    
                    # update alphas_i with the updated alphas_j+
                    self.alphas[i] += y[i] * y[j] * (alpha_j_old - self.alphas[j])


                    # Compute and update new bias
                    b1 = self.b - E_i - y[i] * (self.alphas[i] - alpha_i_old) * self.K[i, i] - y[j] * (self.alphas[j] - alpha_j_old) * self.K[i, j]
                    b2 = self.b - E_j - y[i] * (self.alphas[i] - alpha_i_old) * self.K[i, j] - y[j] * (self.alphas[j] - alpha_j_old) * self.K[j, j]

                    if 0 < self.alphas[i] < self.C:
                        self.b = b1
                    elif 0 < self.alphas[j] < self.C:
                        self.b = b2
                    else:
                        self.b = (b1 + b2) / 2.0


                    num_changed_alphas += 1

                # Check for convergence, so if no alphas changed update passes if alpha was updated reset num of passes
                if num_changed_alphas == 0:
                    passes += 1
                else:
                    passes = 0


            self.support_vectors_ = self.X[self.alphas > 1e-5]
            self.support_vector_labels_ = self.y[self.alphas > 1e-5]
            self.alphas_ = self.alphas[self.alphas > 1e-5]


    def _decision_func_single(self, x):
        result = 0
        for alpha, sv_y, sv in zip(self.alphas, self.y, self.X):
            result += alpha * sv_y * rbf_kernel(x, sv, self.gamma)
        return result + self.b

    def predict(self, X):
        y_pred = np.array([self._decision_func_single(x) for x in X])
        return np.sign(y_pred)

In [9]:
C = 1.0
gamma = 0.5
svm = svm_rbf(C=C, gamma=gamma)
print("\nTraining the SVM model...")
svm.fit(X_train, y_train)
print("Training completed.")
num_sv = len(svm.alphas_)
print(f"Number of support vectors: {num_sv}")

# Step 6: Make Predictions
print("\nMaking predictions on the test set...")
predictions = svm.predict(X_test)

# Step 7: Evaluate the Model
accuracy = np.mean(predictions == y_test)
print(f"\nModel Accuracy on Test Set: {accuracy * 100:.2f}%")

def confusion_matrix(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    tn = np.sum((y_true == -1) & (y_pred == -1))
    fp = np.sum((y_true == -1) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == -1))
    return np.array([[tp, fp],
                     [fn, tn]])

cm = confusion_matrix(y_test, predictions)
print("\nConfusion Matrix:")
print("TP:", cm[0,0], " FP:", cm[0,1])
print("FN:", cm[1,0], " TN:", cm[1,1])

precision = cm[0,0] / (cm[0,0] + cm[0,1] + 1e-10)
recall = cm[0,0] / (cm[0,0] + cm[1,0] + 1e-10)
f1_score = 2 * precision * recall / (precision + recall + 1e-10)

print(f"\nPrecision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1_score:.2f}")


Training the SVM model...
Training completed.
Number of support vectors: 0

Making predictions on the test set...

Model Accuracy on Test Set: 97.28%

Confusion Matrix:
TP: 0  FP: 0
FN: 0  TN: 0

Precision: 0.00
Recall: 0.00
F1-Score: 0.00
